# 跑单一reward

In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from DataLoader.DataLoader import DataLoader
from DataLoader.DataBasedAgent import DataBasedAgent
from DataLoader.DataRLAgent import DataRLAgent
import DeepRLAgent.VanillaInput.Train as Train
from PatternDetectionInCandleStick.Evaluation import Evaluation
import distinctipy

from importlib import reload

Train = reload(Train)
DeepRL = Train.Train
from utils_best_arm import add_train_portfo, add_test_portfo, plot_return, calc_return, plot_action_point, setup_logger
pd.options.display.max_colwidth = 100

device = "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CURRENT_PATH = os.getcwd()

In [2]:
def train(
    DATASET_NAME, 
    split_point='2018-01-01', 
    begin_date='2010-01-01', 
    end_date='2020-08-24', 
    model_start_date="",
    model_end_date="",
    initial_investment=1000,
    transaction_cost=0.0001,
    load_from_file=True,
    reward_type="profit",
    seed=42, 
    state_mode=1,
    n_episodes=5,
    lamb=0.0001,
    GAMMA=0.7, 
    n_step=5, 
    BATCH_SIZE=10, 
    ReplayMemorySize=20,
    TARGET_UPDATE=5,
    window_size=None, 
    train_portfolios={},
    test_portfolios={},
    arms=[],
    show_all = False,
    ratio_threshold=0.9,
):
    data_loader = DataLoader(DATASET_NAME, split_point=split_point, begin_date=begin_date, end_date=end_date, load_from_file=load_from_file)
    
    dataTrain_agent = DataRLAgent(data_loader.data_train, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    dataTest_agent = DataRLAgent(data_loader.data_test, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    agent = DeepRL(data_loader, dataTrain_agent, dataTest_agent, 
                DATASET_NAME,  state_mode, window_size, transaction_cost,
                BATCH_SIZE=BATCH_SIZE, GAMMA=GAMMA, ReplayMemorySize=ReplayMemorySize,
                TARGET_UPDATE=TARGET_UPDATE, n_step=n_step, arms=arms)
    
    path = f"./Results/{DATASET_NAME}/{model_start_date}~{model_end_date}/{seed}/train"
    arm = arms[0]
    name = f'{arm["name"]}_{arm["lamb"]}'
    model_path = f"{path}/model_{name}_{seed}.pkl"
    agent_test = agent.test(initial_investment=initial_investment, test_type='test', model_path=model_path)
    test_portfolio = agent_test.get_daily_portfolio_value()
    test_portfolio = pd.Series(test_portfolio).pct_change(1).fillna(0).values.tolist() 
    model_name = f'DQN-stock:{DATASET_NAME}-reward:{name}-seed:{seed}'
    return data_loader, {"name": model_name, "portfo": test_portfolio}

In [7]:
initial_investment = 1000


kwargs = {
    "load_from_file": True, 
    "transaction_cost": 0.0000,
    "initial_investment": initial_investment,
    "state_mode": 1,
    "GAMMA": 0.7, 
    "n_step": 5, 
    "BATCH_SIZE": 10, 
    "ReplayMemorySize": 20,
    "TARGET_UPDATE": 5,
    "window_size": None, 
    "lamb": 0.0,
}

_file = "AAPL"

_begin_date = '20{}-01-01'
_end_date = '20{}-01-01'
_split_point = '20{}-01-01' 

arms = [
    { "name": "profit", "lamb": 0},
    { "name": "regularized", "lamb": 0.01},
    { "name": "regularized", "lamb": 0.05},
    { "name": "regularized", "lamb": 0.1},
    { "name": "regularized", "lamb": 0.2},
]

dates = [
    ("2019-06-23", "2020-06-22", "2016-01-01", "2019-01-01"),
    ("2020-06-22", "2021-06-22", "2017-01-01", "2020-01-01"),
    ("2021-06-23", "2022-06-23", "2018-01-01", "2021-01-01"),
]

random_seeds = 100
results = []
portfolios_saved = {}
files = sorted(os.listdir("./Data/"))
for _file in files[:]:
    print(_file)
    portfolios_saved[_file] = {}

    for idx, arm in enumerate(arms[:]):
        results2 = []
        portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"] = []
        
        for seed in tqdm(range(random_seeds)):
            
            ls = []
            bhs = []
        
            train_portfolios = {}
            test_portfolios = {}
            tmp_result = []
            
            for date in dates:
                model_start_date = date[2]
                model_end_date = date[3]
                split_point = date[0]
                end_date = date[1]

                kwargs.update({
                    "begin_date": "2016-01-01", 
                    "end_date": end_date, 
                    "split_point": split_point,
                    "model_start_date": model_start_date,
                    "model_end_date": model_end_date,
                    "DATASET_NAME": _file,
                    "reward_type": "",
                    "seed": seed,
                    "n_episodes": 20,
                    "arms": [arm],
                    "show_all": True,
                    "ratio_threshold": 3,
                    "train_portfolios": train_portfolios,
                    "test_portfolios": test_portfolios,
                })

                data_loader, model = train(**kwargs)
                ls.extend(model["portfo"])
                bh = data_loader.data_test_with_date["close"]
                bhs.append(bh)
                tmp_result.extend(model["portfo"])

            add_test_portfo(test_portfolios, seed, ls)
            if seed == 0: 
                bhs = pd.concat(bhs, axis=0)
                bh_percentage = bhs.pct_change(1).fillna(0).values
                add_test_portfo(test_portfolios, 'B&H', bh_percentage)
            indexes = calc_return(bh_percentage, test_portfolios)
            results2.append(indexes)
            portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"].append(tmp_result)

        # path = f"./Results/{_file}/exp4_concat"
        # if not os.path.exists(path):
        #     os.mkdir(path)
        
        # save_path = f"{path}/{arm['name']}-{arm['lamb']}.csv"

        # portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"].insert(0, bh_percentage.tolist())
        # _df_ = pd.DataFrame(portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"]).T.fillna(0)
        # _df_.to_csv(save_path, index=False)

        results2_df = pd.concat(results2, axis=1)
        results2_bh = results2_df["B&H"]
        del results2_df["B&H"]
        final = pd.concat([
            results2_bh,
            results2_df.median(axis=1)
        ], axis=1)
        final.columns = [f"{_file}-B&H", f"{_file}-{arm['name']}-{arm['lamb']}"]
        if idx > 0:
            del final[f"{_file}-B&H"]
        results.append(final)


AAPL


100%|██████████| 100/100 [02:12<00:00,  1.32s/it]


AMGN


100%|██████████| 100/100 [02:15<00:00,  1.36s/it]


AXP


...

100%|██████████| 100/100 [01:07<00:00,  1.48it/s]


WBA


100%|██████████| 100/100 [01:09<00:00,  1.44it/s]


WMT


100%|██████████| 100/100 [01:07<00:00,  1.49it/s]


In [10]:
def output_bh(symbol):
    raw_df = pd.read_csv(f"./Data/{symbol}/{symbol}.csv")
    raw_df = raw_df[["Date", "Close"]]
    raw_df["pct"] = raw_df["Close"].pct_change(1)
    raw_df = raw_df.query("Date >= '2016-01-01'")
    raw_df = raw_df.set_index("Date")
    del raw_df["Close"]
    raw_df = raw_df.reset_index()
    dates = [
        ("2019-06-23", "2020-06-22", "2016-01-01", "2019-01-01"),
        ("2020-06-22", "2021-06-22", "2017-01-01", "2020-01-01"),
        ("2021-06-23", "2022-06-23", "2018-01-01", "2021-01-01"),
    ]
    ls = []
    for date in dates:
        date1, date2, _, _ = date
        ls.append(raw_df.query(f"Date >= '{date1}' & Date <= '{date2}'"))
    bh = pd.concat(ls)
    bh.reset_index(inplace=True)
    del bh["index"]
    return bh

symbol = "AAPL"
# output_bh(symbol)

In [13]:
symbols = list(portfolios_saved.keys())[:]
ls = []
for symbol in symbols:
    rewards = ['profit-0', "regularized-0.01", "regularized-0.05", "regularized-0.1", "regularized-0.2",]
    # NOTE 这个是获取BH，因为每一个reward的第一列都是BH
    bh = output_bh(symbol)
    date_ls = bh["Date"].tolist()
    # tmp = ((1 + bh["pct"]).cumprod() - 1)
    tmp = bh["pct"]
    tmp.name = f"{symbol}-B&H"

    tmp = tmp.shift(1)
    tmp = tmp.fillna(0)
    ls.append(tmp)

    for reward in rewards:
        # tmp = pd.DataFrame(portfolios_saved[symbol][reward]).T.median(axis=1)
        tmp = ((1 + pd.DataFrame(portfolios_saved[symbol][reward]).T).cumprod() - 1).median(axis=1)
        if reward == "profit-0": reward = "FP5"
        if reward == "regularized-0.01": reward = "FPR-0.01"
        if reward == "regularized-0.05": reward = "FPR-0.05"
        if reward == "regularized-0.1": reward = "FPR-0.1"
        if reward == "regularized-0.2": reward = "FPR-0.2"
        tmp.name = symbol + "-" + reward
        ls.append(tmp)


results_cumreturn = pd.concat(ls, axis=1)
for col in results_cumreturn.columns:
    if results_cumreturn[col].isna().sum() > 0:
        results_cumreturn[col] = results_cumreturn[col].shift(results_cumreturn[col].isna().sum())


results_cumreturn = results_cumreturn.iloc[3:]
results_cumreturn["date"] = date_ls
results_cumreturn = results_cumreturn.set_index("date")
# results_cumreturn.to_csv("./ts-run-results/[exp3]single-returns.csv",)
results_cumreturn.to_csv("./ts-run-results/[exp3]single-cumreturns.csv")
results_cumreturn

,AAPL-B&H,AAPL-FP5,AAPL-FPR-0.01,AAPL-FPR-0.05,AAPL-FPR-0.1,AAPL-FPR-0.2,AMGN-B&H,AMGN-FP5,AMGN-FPR-0.01,AMGN-FPR-0.05,...,WBA-FPR-0.01,WBA-FPR-0.05,WBA-FPR-0.1,WBA-FPR-0.2,WMT-B&H,WMT-FP5,WMT-FPR-0.01,WMT-FPR-0.05,WMT-FPR-0.1,WMT-FPR-0.2
date,,,,,,,,,,,,,,,,,,,,,
2019-06-24,0.000000,0.021322,0.021322,0.021322,0.021322,0.0,0.000000,0.000000,0.000000,0.0,...,0.029651,0.000000,0.000000,0.0,0.000000,-0.005600,-0.005600,0.000000,0.0,0.0
2019-06-25,-0.001006,0.012016,0.012016,0.012016,0.012016,0.0,-0.012185,0.001413,0.000000,0.0,...,0.032484,0.000000,0.000000,0.0,0.000990,-0.002077,-0.002077,0.003542,0.0,0.0
2019-06-26,-0.015158,0.030577,0.030577,0.030577,0.012016,0.0,-0.002110,0.012662,0.000000,0.0,...,0.032484,0.000000,0.000000,0.0,-0.004675,-0.000903,-0.000903,0.004723,0.0,0.0
2019-06-27,0.021629,0.036611,0.036611,0.036611,0.017941,0.0,-0.006886,0.014238,0.000000,0.0,...,0.032484,0.000000,0.000000,0.0,-0.005058,0.007948,0.007948,0.013072,0.0,0.0
2019-06-28,-0.000300,0.045201,0.036611,0.036611,0.017941,0.0,0.004640,0.014238,0.000000,0.0,...,0.032484,0.000000,0.000000,0.0,-0.000545,0.014451,0.014451,0.013624,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-16,0.020111,1.836259,2.079193,2.450024,1.225393,0.0,-0.000509,2.256941,1.902073,0.0,...,4.876137,2.691480,0.020019,0.0,-0.000670,1.094260,1.505609,0.494634,0.0,0.0
2022-06-17,-0.039651,1.866717,2.120153,2.500784,1.229870,0.0,-0.020672,2.308143,1.947696,0.0,...,4.992703,2.764708,0.020019,0.0,0.010387,1.162953,1.577417,0.494634,0.0,0.0
2022-06-21,0.011533,1.901121,2.145046,2.530448,1.244158,0.0,0.017381,2.332148,1.969086,0.0,...,5.045009,2.797568,0.020019,0.0,-0.019317,1.136751,1.546193,0.485564,0.0,0.0


In [18]:
results_df = pd.concat(results, axis=1)
# results_df.to_csv(f"./ts-run-results/[{random_seeds}]ts-single_reward.csv")

def find(symbol):
    ls = []
    for col in results_df.columns:
        if symbol in col:
            ls.append(col)

    return ls

files = os.listdir("./Results/")
ls = find(files[0])
results_df[ls]

,AAPL-B&H,AAPL-profit-0,AAPL-regularized-0.01,AAPL-regularized-0.05,AAPL-regularized-0.1,AAPL-regularized-0.2,AAPL-sharpe-0.01,AAPL-volatility-10
sortino_test,1.617304,1.732869,1.880912,2.392482,3.530022,0.0,1.561169,1.347871
sharpe_test,1.170390,1.286437,1.391242,1.731692,1.856541,0.0,1.134080,0.975010
risk_test,0.350629,0.312165,0.304180,0.252178,0.164556,0.0,0.348187,0.247231
mdd_test,0.314273,0.354480,0.345128,0.322186,0.107889,0.0,0.314273,0.314273
downrisk_test,0.253739,0.231492,0.224200,0.181733,0.083222,0.0,0.253328,0.187509
cumreturn_test,1.853616,1.924140,2.162058,2.563134,1.251909,0.0,1.730817,0.784039


In [5]:
import pandas as pd
df = pd.read_csv("./ts-run-results/[exp3]single-cumreturns.csv")
df

,date,AAPL-B&H,AAPL-FP5,AAPL-FPR-0.01,AAPL-FPR-0.05,AAPL-FPR-0.1,AAPL-FPR-0.2,AAPL-SR-0.01,AAPL-Vol-10,AMGN-B&H,...,DIS-SR-0.01,DIS-Vol-10,GS-B&H,GS-FP5,GS-FPR-0.01,GS-FPR-0.05,GS-FPR-0.1,GS-FPR-0.2,GS-SR-0.01,GS-Vol-10
0,2019-06-24,0.000000,0.021322,0.021322,0.021322,0.021322,0.0,0.021322,0.000000,0.000000,...,-0.004573,0.000000,0.000000,0.016627,0.016627,0.0,0.0,0.0,0.016627,0.000000
1,2019-06-25,-0.001006,0.012016,0.012016,0.012016,0.012016,0.0,0.012016,0.000000,-0.012185,...,-0.002144,0.000000,0.007911,0.043558,0.043558,0.0,0.0,0.0,0.043558,0.000000
2,2019-06-26,-0.016148,0.030577,0.030577,0.030577,0.012016,0.0,0.030577,0.012016,-0.014270,...,0.012219,0.000000,0.000613,0.055085,0.055085,0.0,0.0,0.0,0.055085,0.000000
3,2019-06-27,0.005131,0.036611,0.036611,0.036611,0.017941,0.0,0.036611,0.017941,-0.021057,...,0.018508,0.000000,0.005461,0.050597,0.055085,0.0,0.0,0.0,0.050597,0.000000
4,2019-06-28,0.004829,0.045201,0.036611,0.036611,0.017941,0.0,0.045201,0.017941,-0.016514,...,0.021723,0.002384,0.017250,0.050903,0.055085,0.0,0.0,0.0,0.050903,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,2022-06-16,1.865246,1.836259,2.079193,2.450024,1.225393,0.0,1.612160,0.763763,0.348826,...,-0.338429,-0.195395,0.593951,2.449534,2.665033,0.0,0.0,0.0,0.466852,0.007527
754,2022-06-17,1.751635,1.866717,2.120153,2.500784,1.229870,0.0,1.694702,0.768637,0.320943,...,-0.345792,-0.199891,0.565981,2.511426,2.730791,0.0,0.0,0.0,0.493170,0.007527
755,2022-06-21,1.783370,1.901121,2.145046,2.530448,1.244158,0.0,1.684389,0.765301,0.343902,...,-0.344319,-0.198992,0.537462,2.508097,2.727255,0.0,0.0,0.0,0.491755,0.007527
756,2022-06-22,1.874555,1.924140,2.171937,2.463739,1.207115,0.0,1.730817,0.784039,0.365029,...,-0.338709,-0.201837,0.565047,2.527104,2.611989,0.0,0.0,0.0,0.500301,0.005001


In [ ]:
df[""]